# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [3]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [5]:
# set up environment
# Load environment variables in a file called .env

load_dotenv(override=True)      # env 파일로드
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [6]:
system_prompt = "you are a assistant which specializes in coding. \
You have to explain code to south korea people.\ try to explain code as korean and easy word \
to who are not major in computer science. Respond in markdown."

In [7]:
# 코드 리팩토링 

def get_client(model: str) -> OpenAI:
    if model == MODEL_GPT:
        return OpenAI()
    # ollama(OpenAI 호환) 클라이언트
    return OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [8]:
def ai_answer(system_prompt: str, user_prompt: str, model: str):
    client = get_client(model)
    stream = client.chat.completions.create(
        model=model,
        messages=[{"role":"system","content":system_prompt},
                  {"role":"user","content":user_prompt}],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        delta = getattr(chunk.choices[0].delta, "content", None) or ""
        response += delta
        # Markdown은 굳이 제거할 필요 없음 (코드블록 표시에 필요)
        update_display(Markdown(response), display_id=display_handle.display_id)

In [9]:
user_prompt = input("프롬프트를 입력하세요: ")

프롬프트를 입력하세요:  Please explain what this code does and why: yield from {book.get("author") for book in books if book.get("author")}


In [10]:
ai_answer(system_prompt, user_prompt, MODEL_GPT)

안녕하세요! 이 코드에 대해 쉽게 설명해드릴게요. 

코드는 파이썬에서 사용되는 것으로, 주로 데이터를 처리할 때 많이 사용됩니다. 이 코드를 여러 부분으로 나누어 설명해볼게요.

### 코드 설명

1. **리스트 컴프리헨션**:
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   - 이 부분은 **집합(set)** 컴프리헨션입니다. `books`라는 리스트에 있는 각각의 `book`에 대해, 각 책의 **저자(author)**를 가져옵니다.
   - `book.get("author")`는 `book`에서 저자를 찾는 방법입니다. 저자가 없다면 `None`을 반환하는데, 이런 경우는 넘어갑니다.
   - `if book.get("author")`는 저자가 있는 경우에만 해당 저자를 포함합니다.
   - 그래서 이 부분은 저자가 있는 책들로부터 저자들의 집합을 만들어줍니다.

2. **yield from**:
   ```python
   yield from ...
   ```
   - `yield from`은 **제너레이터(generator)**를 만들 때 사용되는 구문입니다. 이렇게 하면 저자들을 하나씩 반환할 수 있습니다.
   - 즉, 저자들의 리스트를 생성하고, 이를 호출한 쪽에서 하나씩 사용할 수 있도록 합니다.

### 전체적 의미

결국 이 코드는 `books` 리스트에 있는 책들의 저자들 중에서, 저자가 있는 경우에 한해 저자들의 집합을 생성하고, 그 집합의 저자들을 하나씩 반환하는 역할을 합니다.

#### 요약
- 주어진 `books`라는 리스트에서 저자가 있는 책들의 저자를 모아서,
- 그 저자들을 하나씩 제공해주는 기능을 하는 코드입니다.

이해가 되셨나요? 추가로 궁금한 점이 있으면 언제든지 물어보세요! 😊

In [11]:
ai_answer(system_prompt, user_prompt, MODEL_LLAMA)

# Explanation of Code: 

`.yield from {...}``

`.get()` is a method used to access the value of an item inside `Hashset` or dictionary.

### what does `yield from {...}` do?

`yield from {...}` is used when you want to call another `iterator` like function and put all values in a new one . It's called to **lazy evaluation** meaning that `yield()` will run until the end, then return.

`.get("author")` will get the book author

**so what the whole expression does?**

The entire expression is used when you have many books with `author` and don't want it to execute on each page. This saves  performance. Let's assume we are doing query in a library, like which ones were written by Lee Do Hyun 

So we call `getAuthor()` from each book and print the author but only for those where writer is `Lee Do Hyun`.